<a href="https://colab.research.google.com/github/jacobiIdentity/AtCoder_Simple_Runtime/blob/main/AtCoder_Simple_CodeTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. おまじない(最初にまとめて実行)

テストコード取得関数

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import json

def fetch_test_cases(url, cache_dir=".cache"):
    os.makedirs(cache_dir, exist_ok=True)
    cache_file = os.path.join(cache_dir, url.replace("https://", "").replace("/", "_") + ".json")

    # キャッシュがあればそれを返す
    if os.path.exists(cache_file):
        with open(cache_file, "r", encoding="utf-8") as f:
            return json.load(f)

    # なければ取得
    resp = requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    samples = []
    h3_tags = soup.find_all("h3")

    for h3 in h3_tags:
        if "入力例" in h3.text:
            input_text = h3.find_next("pre").text
            output_h3 = h3.find_next(lambda tag: tag.name == "h3" and "出力例" in tag.text)
            if output_h3:
                output_text = output_h3.find_next("pre").text
                samples.append((input_text, output_text))

    samples = [(inp.replace("\r", ""), out.replace("\r", "")) for inp, out in samples]

    # キャッシュ保存
    with open(cache_file, "w", encoding="utf-8") as f:
        json.dump(samples, f, ensure_ascii=False, indent=2)

    return samples


テスト実行関数(1回)

In [2]:
import subprocess
import time
import sys
import io

def run_external_test(command, input_text="", timeout_sec=10):
    try:
        start = time.perf_counter()
        result = subprocess.run(
            command,
            input=input_text,
            text=True,
            capture_output=True,
            timeout=timeout_sec,
            check=False  # ← 例外は出さない
        )
        end = time.perf_counter()
        elapsed = end - start
    except subprocess.TimeoutExpired:
        print("⚠️ タイムアウト発生")
        return None,None
    except FileNotFoundError:
        print("⚠️ コマンドが見つかりません")
        return None,None

    # 終了コードで OK/NG 判定
    if result.returncode != 0:
        print(f"⚠️ エラー終了 (returncode={result.returncode})")
        print("標準エラー:", result.stderr.strip())

    # 標準出力は返す
    return result.stdout.strip(),elapsed

# --- 使用例 ---
# output = run_external_test(["python3", "main.py"], input_text="5 15\n3 1 4 1 5\n")
# print("出力:", output)


In [3]:
def run_all():
    samples = fetch_test_cases(problem_url)

    results = []
    times = []

    for idx, (inp, out) in enumerate(samples, 1):
        print(f"■ sample{idx}:")
        expected = out.rstrip("\n")
        executed, elapsed = run_external_test(["python3", "main.py"], input_text=inp)

        if executed is None:
            result = "ERROR"
            print("<result> 実行失敗")
        elif expected == executed:
            result = "AC"
            print("<result> AC!")
        else:
            result = "NG"
            print("<result> ng...\n")
            print("input:")
            print(inp)
            print("output:")
            print(executed)
            print("expected:")
            print(expected)

        if elapsed is not None:
            times.append(elapsed)
            print(f"<time> {elapsed:.6f} 秒")

        results.append(result)
        print()

    # --- まとめ ---
    total = len(results)
    ac_count = results.count("AC")
    ng_count = results.count("NG")
    err_count = results.count("ERROR")

    print("=" * 20)
    print(f"✅ AC: {ac_count} / {total}")
    print(f"❌ NG: {ng_count} / {total}")
    if err_count:
        print(f"⚠️ 実行失敗: {err_count}")

    if times:
        avg_time = sum(times) / len(times)
        max_time = max(times)
        print(f"平均実行時間: {avg_time:.6f} 秒")
        print(f"最大実行時間: {max_time:.6f} 秒")
    print("=" * 20)


# 1. urlを入力

In [4]:
problem_url = "https://atcoder.jp/contests/abc424/tasks/abc424_a"

# 2. コードを入力 (二行目から書いてね)

In [5]:
%%writefile main.py
#!/usr/bin/env python3
a,b,c = map(int,input().split())
print('YNeos'[not(a==b or b==c or c==a)::2])

Writing main.py




# 3. テスト実行

In [6]:
run_all()

■ sample1:
<result> AC!
<time> 0.463263 秒

■ sample2:
<result> AC!
<time> 0.270619 秒

■ sample3:
<result> AC!
<time> 0.424331 秒

✅ AC: 3 / 3
❌ NG: 0 / 3
平均実行時間: 0.386071 秒
最大実行時間: 0.463263 秒
